[<< [Functional Programming and OOP Intersection](./08_functional_programming_and_oop.ipynb) | [Index](./00_index.ipynb) | >>]

There is a great [article with visualization for Functors, Applicative and Monad](https://www.adit.io/posts/2013-04-17-functors,_applicatives,_and_monads_in_pictures.html)

[![Monad](https://img.youtube.com/vi/e6tWJD5q8uw/0.jpg)](https://www.youtube.com/watch?v=e6tWJD5q8uw)

## Functors

- A [Functor](https://en.wikipedia.org/wiki/Functor) in functional programming is a type that implements a map operation, which applies a function to a wrapped value.
- It's a way to apply a function over or around some structure that we don’t want to alter, like a list, Maybe, or a Tree.
- Functors must obey two laws: identity (if we map the id function over a functor, the functor that we get back should be the same as the original functor) and composition (composing two functions and then mapping the resulting function over a functor should be the same as first mapping one function over the functor and then mapping the other one).
- They are used to abstract over data types that can be mapped over, and provide a consistent interface for applying functions to values in a context.

In [5]:
# All iterators are funtors
numbers = [1, 2, 3, 4, 5]

# identity (if we map the id function over a functor, the functor that we get back should be the same as the original functor)
identity = lambda num: num
assert list(map(identity, numbers)) == numbers

# composition (composing two functions and then mapping the resulting function over a functor should be the same as first mapping one function over the functor and then mapping the other one)
increment = lambda num: num + 1
square = lambda num: num ** 2

# map(f(g(x)))
value1 = map(lambda num: square(increment(num)), numbers)
# map(f, map(g(x)))
value2 = map(square, map(increment, numbers))
assert list(value1) == list(value2)

## Applicative Functors

- An [Applicative Functor](https://en.wikipedia.org/wiki/Applicative_functor) is a type of functor that allows for function application within a computational context.
- They provide a structure for applying a function wrapped in a context to a value wrapped in a context (or another function wrapped in a context).
- Applicative functors must obey two laws: identity (applying the pure id function to a value should give the same value) and homomorphism (applying a function to a value should be the same as applying a function to a value and then applying pure).
- Applicative functors are more powerful than regular functors, and are used when values of computations depend on each other.
- They are a fundamental concept in languages like Haskell, and are used in other languages like JavaScript and Python through libraries.

## Monad

- A [Monad](https://en.wikipedia.org/wiki/Monad_(functional_programming)) in functional programming is a design pattern that allows function sequencing and defines how functions, actions, inputs, and outputs can be used together.
- It's a type of computational structure that encapsulates operations over elements while providing a way to chain these operations together.
- Monads have three primary properties: unit (or return), bind (or >>=), and associativity.
- They are used to handle side-effectful operations, encapsulate I/O actions, manage state, handle exceptions, and more.
- Monads are a fundamental part of Haskell and have been adapted for use in other languages like JavaScript and Python.

[![Monad](https://img.youtube.com/vi/t1e8gqXLbsU/0.jpg)](https://www.youtube.com/watch?v=t1e8gqXLbsU)

**Need for Monad?**
(Showing example of `Maybe` monad, because I have found use case of this while working)

In [1]:
from dataclasses import dataclass, field
from typing import List, Optional


@dataclass
class Employee:
    name: str


@dataclass
class Team:
    name: str
    employees: List[Employee] = field(default_factory=list)

    def add_employee(self, employee: Employee):
        self.employees.append(employee)


@dataclass
class Department:
    name: str
    teams: List[Team] = field(default_factory=list)

    def add_team(self, team: Team):
        self.teams.append(team)

    def get_team(self, team_name: str) -> Optional[Team]:
        for team in self.teams:
            if team.name == team_name:
                return team
        return None


@dataclass
class Company:
    name: str
    departments: List[Department] = field(default_factory=list)

    def add_department(self, department: Department):
        self.departments.append(department)

    def get_department(self, department_name: str) -> Optional[Department]:
        for department in self.departments:
            if department.name == department_name:
                return department
        return None


# Create company
intel = Company("Intel")

# Create departments
engineering = Department("Engineering")
hr = Department("HR")

# Create teams
design = Team("Design")
testing = Team("Testing")

# Create employees
ramesh = Employee("Ramesh Kumar")
sunita = Employee("Sunita Sharma")
priya = Employee("Priya Singh")
raj = Employee("Raj Kapoor")
anil = Employee("Anil Gupta")
deepak = Employee("Deepak Verma")

# Add employees to teams
design.add_employee(ramesh)
design.add_employee(sunita)
testing.add_employee(priya)
testing.add_employee(raj)

# Add teams to departments
engineering.add_team(design)
engineering.add_team(testing)

# Add employees directly to HR department (no team)
general_hr = Team("General HR")  # Creating a general HR team to hold HR employees
general_hr.add_employee(anil)
general_hr.add_employee(deepak)
hr.add_team(general_hr)

# Add departments to company
intel.add_department(engineering)
intel.add_department(hr)


def print_company_structure(company):
    print(f"Company: '{company.name}'")
    for department in company.departments:
        print(f"   - Department: '{department.name}'")
        for team in department.teams:
            print(f"     - Team: '{team.name}'")
            for employee in team.employees:
                print(f"       - Employee: '{employee.name}'")


print_company_structure(intel)

Company: 'Intel'
   - Department: 'Engineering'
     - Team: 'Design'
       - Employee: 'Ramesh Kumar'
       - Employee: 'Sunita Sharma'
     - Team: 'Testing'
       - Employee: 'Priya Singh'
       - Employee: 'Raj Kapoor'
   - Department: 'HR'
     - Team: 'General HR'
       - Employee: 'Anil Gupta'
       - Employee: 'Deepak Verma'


In [2]:
def get_employees(company, department_name, team_name):
    if company is not None:
        department = company.get_department(department_name)
        if department is not None:
            team = department.get_team(team_name)
            if team is not None:
                return team.employees
    return None

In [3]:
print(f"{get_employees(company=intel, department_name='Engineering', team_name='Testing') = }")
print(f"{get_employees(company=intel, department_name='HR', team_name='General HR') = }")
print()
print(f"{get_employees(company=intel, department_name='HR', team_name='Testing') = }")
print(f"{get_employees(company=intel, department_name='Engineering', team_name='General HR') = }")

get_employees(company=intel, department_name='Engineering', team_name='Testing') = [Employee(name='Priya Singh'), Employee(name='Raj Kapoor')]
get_employees(company=intel, department_name='HR', team_name='General HR') = [Employee(name='Anil Gupta'), Employee(name='Deepak Verma')]

get_employees(company=intel, department_name='HR', team_name='Testing') = None
get_employees(company=intel, department_name='Engineering', team_name='General HR') = None


In [4]:
from returns.maybe import Maybe


def get_employees(company, department_name, team_name):
    return (
        Maybe.from_optional(company)
        .bind_optional(lambda valid_company: valid_company.get_department(department_name))
        .bind_optional(lambda valid_department: valid_department.get_team(team_name))
        .bind_optional(lambda valid_team: valid_team.employees)
    )

In [5]:
print(f"{get_employees(company=intel, department_name='Engineering', team_name='Testing') = }")
print(f"{get_employees(company=intel, department_name='HR', team_name='General HR') = }")
print()
print(f"{get_employees(company=intel, department_name='HR', team_name='Testing') = }")
print(f"{get_employees(company=intel, department_name='Engineering', team_name='General HR') = }")

get_employees(company=intel, department_name='Engineering', team_name='Testing') = <Some: [Employee(name='Priya Singh'), Employee(name='Raj Kapoor')]>
get_employees(company=intel, department_name='HR', team_name='General HR') = <Some: [Employee(name='Anil Gupta'), Employee(name='Deepak Verma')]>

get_employees(company=intel, department_name='HR', team_name='Testing') = <Nothing>
get_employees(company=intel, department_name='Engineering', team_name='General HR') = <Nothing>


[<< [Functional Programming and OOP Intersection](./08_functional_programming_and_oop.ipynb) | [Index](./00_index.ipynb) | >>]
